In [1]:
import spacy
import os
import time
from spacy.matcher import PhraseMatcher
from spacy import displacy
nlp = spacy.load("C:\\Users\\georgmerz\\AppData\\Local\\Continuum\\anaconda3\\Lib\\site-packages\\de_core_news_sm\\de_core_news_sm-2.0.0")

In [9]:
import speech_recognition as sr


# obtain audio from the microphone
r = sr.Recognizer()
with sr.Microphone() as source:
    r.adjust_for_ambient_noise(source)
    print("Say something")
    audio = r.listen(source)
print("Audiotake finished")

# recognize speech using Google Speech Recognition
try:
    # for testing purposes, we're just using the default API key
    # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
    # instead of `r.recognize_google(audio)`
    doc_string=r.recognize_google(audio,language="de")
except sr.UnknownValueError:
    print("Google Speech Recognition could not understand audio")
    output = "Ich habe dich leider nicht verstanden"
except sr.RequestError as e:
    print("Could not request results from Google Speech Recognition service; {0}".format(e))
    output = "Dieser Dienst ist zur Zeit leider nicht verfügbar."
else:
    
    print(doc_string)
    startbahnhof=["Von","von","Ab","ab"]
    zielbahnhof=["nach","zu","Nach","Zu"]

    doc=nlp(doc_string)
    token_list=[word for word in doc]
    ents = doc.ents 
    for ent in ents:
        print(ent.label_)

    locations = list(filter(lambda x: x.label_ == "LOC", ents))

    print(locations)




    for location in locations:
        doc_string=doc_string.replace(location.text,"Frankfurt")

    proc_doc=nlp(doc_string)
    matcher = PhraseMatcher(nlp.vocab)
    matcher.add('Frankfurt', None, nlp("Frankfurt"))
    matches=matcher(proc_doc)
    indices=[match[1] for match in matches]

    print("Start incredible graph search...")

    destination_list=[]
    start_list=[]
    for k in range(len(locations)):
        tok_word=proc_doc[indices[k]]


        while(True):
            if(tok_word.head.text in startbahnhof):
                start_list.append(locations[k])
                break
            if(tok_word.head.text in zielbahnhof):
                destination_list.append(locations[k])
                break
            if(tok_word==tok_word.head):
                break
            tok_word=tok_word.head


    if len(start_list) > 0 and len(destination_list) > 0:  
        import schiene
        s = schiene.Schiene()
        fahrplan=s.connections(start_list[0].text, destination_list[0].text)
        if len(fahrplan) >0:

            output = "Der nächste Zug von "+str(start_list[0].text)+ " fährt um " +fahrplan[0]["departure"]+    " nach " + str(destination_list[0].text) + " und kommt um " +str(fahrplan[0]["arrival"])+ " Uhr an"
        else: 
            output = "Ich habe leider keine Verbindung für dich gefunden."
    else:
        output = "Ich bin mir nicht ganz sicher, was du wissen willst."
# ENDE ELSE-BLOCK
    
from gtts import gTTS 
  
# This module is imported so that we can  
# play the converted audio 
import os 
  
# The text that you want to convert to audio 

  
# Language in which you want to convert 
language = 'de'
  
# Passing the text and language to the engine,  
# here we have marked slow=False. Which tells  
# the module that the converted audio should  
# have a high speed 
myobj = gTTS(text=output, lang=language, slow=False)
out_filename = "welcome_" + str(int(time.time())) + ".mp3"
print(out_filename)
myobj.save(out_filename) 

from pygame import mixer # Load the required library

mixer.init()
mixer.music.load(out_filename)
mixer.music.play()
time.sleep(15)
mixer.quit()

Say something
Audiotake finished
Hallo ich möchte von Rostock nach Bielefeld
LOC
LOC
[Rostock, Bielefeld]
Start incredible graph search...
welcome_1544280824.mp3


In [12]:
os.system("welcome.mp3")

1